# Índice

1. [Importação de dados e de bibliotecas; separação de cada batch ](#Importação-de-dados-e-de-bibliotecas;-separação-de-cada-batch )
2. [Tipos de controle](#Tipos-de-controle)
3. [Estudo batches 0-29: Controlled by recipe driven approach](#Estudo-batches-0-29:-Controlled-by-recipe-driven-approach)
4. [Estudo batches 60-89: Controlled by an Advanced Process Control (APC)](#Estudo-batches-60-89:-Controlled-by-an-Advanced-Process-Control-(APC))

# Importação de dados e de bibliotecas; separação de cada batch 

In [ ]:
import numpy as np
import pandas as pd

import sys, os

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.offline import iplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import metrics

from tensorflow import keras
from keras import layers

import tensorflow as tf
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, Dense, RNN, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.ops import math_ops

import time

from tensorflow.random import set_seed

import tensorflow_addons as tfa
from tensorflow_addons.rnn import ESNCell

In [ ]:
data = pd.read_csv("100_Batches_IndPenSim_V3.csv")

# ou: data = pd.read_csv(r"Especifique aqui o caminho para o arquivo .csv que contém os dados referentes às bateladas simuladas no IndPenSim")

In [ ]:
l = data[data['Time (h)']==0.2].index.to_list()
batches_list=[]
for idx in range(0, len(l)):
    if idx+1 <= len(l)-1:
        batches_list.append(data.loc[l[idx]:(l[idx+1]-1)])
    else:
        batches_list.append(data.loc[l[idx]:])

Queremos que todos os batches utilizados na rede neural tenham a mesma duração de 230 horas (é a duração dos batches 29, 17, 5, ...). Os batches com duração menor que essa serão descartados, e, para os batches com duração maior, apenas os dados até esta hora serão considerados.

In [ ]:
duracao_minima = batches_list[29].shape[0]
duracao_minima

In [ ]:
batches_utilizados = []

for i in range(0,30):
    if batches_list[i].shape[0] >= duracao_minima:
        batches_utilizados.append(i)
        batches_list[i] = batches_list[i][:duracao_minima]  # Atualizamos os dados para estes batches em batches_list

batches_utilizados

In [ ]:
colunas_utilizaveis = [
 'Time (h)',
 'Aeration rate(Fg:L/h)',
 'Agitator RPM(RPM:RPM)',
 'Sugar feed rate(Fs:L/h)',
 'Acid flow rate(Fa:L/h)',
 'Base flow rate(Fb:L/h)',
 'Heating/cooling water flow rate(Fc:L/h)',
 'Heating water flow rate(Fh:L/h)',
 'Water for injection/dilution(Fw:L/h)',
 'Air head pressure(pressure:bar)',
 'Dumped broth flow(Fremoved:L/h)',
 'Substrate concentration(S:g/L)',
 'Dissolved oxygen concentration(DO2:mg/L)',
 'Penicillin concentration(P:g/L)',
 'Vessel Volume(V:L)',
 'Vessel Weight(Wt:Kg)',
 'pH(pH:pH)',
 'Temperature(T:K)',
 'Generated heat(Q:kJ)',
 'carbon dioxide percent in off-gas(CO2outgas:%)',
 'PAA flow(Fpaa:PAA flow (L/h))',
 'PAA concentration offline(PAA_offline:PAA (g L^{-1}))',
 'Oil flow(Foil:L/hr)',
 'NH_3 concentration off-line(NH3_offline:NH3 (g L^{-1}))',
 'Oxygen Uptake Rate(OUR:(g min^{-1}))',
 'Oxygen in percent in off-gas(O2:O2  (%))',
 'Offline Penicillin concentration(P_offline:P(g L^{-1}))',
 'Offline Biomass concentratio(X_offline:X(g L^{-1}))',
 'Carbon evolution rate(CER:g/h)',
 'Ammonia shots(NH3_shots:kgs)',
 'Viscosity(Viscosity_offline:centPoise)']

Os batches que fornecem maiores valores finais de conc. de penicilina são os batches 7 e 28. Os que fornecem menores conc. são 8 e 29. Queremos que estes batches sejam utilizados para treinamento do modelo RNN. Os batches intermediários serão separados em treino e teste.  

In [ ]:
ultimos_valores_penic_batches_utilizados = []
for i in batches_utilizados:
    ultimos_valores_penic_batches_utilizados.append(batches_list[i]['Penicillin concentration(P:g/L)'].to_list()[-1])

ultimos_valores_penic_batches_utilizados = pd.DataFrame(ultimos_valores_penic_batches_utilizados, batches_utilizados)
ultimos_valores_penic_batches_utilizados = ultimos_valores_penic_batches_utilizados.sort_values(by = 0, ascending = False)

batches_maior_penic = ultimos_valores_penic_batches_utilizados.index[[0,1]].to_list()
batches_menor_penic = ultimos_valores_penic_batches_utilizados.index[[-1,-2]].to_list()
print(batches_maior_penic)
print(batches_menor_penic)

In [ ]:
import random
rng = np.random.RandomState(42)
batches_utilizados = np.array(batches_utilizados)

proporcao_treino = 0.75
batches_garantidos_no_treino = np.array(batches_maior_penic + batches_menor_penic)

qntd_randomizada_treino = int(  (len(batches_utilizados) * proporcao_treino - len(batches_garantidos_no_treino) )  )
                              
batches_randomizados_treino = rng.choice(np.setxor1d(batches_utilizados, batches_garantidos_no_treino), qntd_randomizada_treino, replace=False) 
# A função np.setxor1d retorna um array com os valores que encontram-se APENAS no primeiro array fornecido. Ou seja, pega o 
# primeiro array e retira os valores que estão no segundo array. https://numpy.org/doc/stable/reference/generated/numpy.setxor1d.html

batches_treino = np.concatenate((batches_randomizados_treino, batches_garantidos_no_treino))

print(len(batches_treino))
batches_treino

In [ ]:
batches_teste = np.setxor1d(batches_utilizados, batches_treino)
print(len(batches_teste))
batches_teste

In [ ]:
batches_treino

In [ ]:
batches_teste

In [ ]:
batches_utilizados

Metodologia utilizada para se treinar a rede neural foi inspirada nos conteúdos do capítulo 5 do livro Neural Networks 
in Bioprocessing and Chemical Engineering, de D. R. Baughman e Y. A. Liu.

In [ ]:
colunas_input = ['Carbon evolution rate(CER:g/h)',
                 'Oxygen Uptake Rate(OUR:(g min^{-1}))', 
                 'Substrate concentration(S:g/L)', 
                 'Penicillin concentration(P:g/L)' ]

qntd_features_input = len(colunas_input)

colunas_output = ['Penicillin concentration(P:g/L)']

colunas_utilizadas = colunas_input

In [ ]:
batches_list[3][colunas_utilizadas]

In [ ]:
for i in batches_utilizados:
    globals()['dados_batch_'+str(i)] = globals()['batches_list'][i].copy().reset_index()[colunas_utilizadas]

# Para fazer o StandardScaling, preciso juntar todos os dados de treino em uma só tabela. Assim, farei o fit para essa tabelona
# e, depois, o transform para todas as tabelas (de treino e de teste) em separado.

# FAZENDO O FIT:

passador = 1

for i in batches_treino:
    
    if passador == 2:
    
        df_1 = globals()['dados_batch_%i' %i]
        df_2 = globals()['dados_batch_%i' %i_anterior]
        
        todos_batches_treino = pd.concat([df_1, df_2], sort=False)

    else:
        
        if passador > 2:
            
            df = globals()['dados_batch_%i' %i]
            todos_batches_treino = pd.concat([todos_batches_treino, df], sort=False)
            
    i_anterior = i                                                                                                                                                                                     
    passador += 1

colunas = todos_batches_treino.columns


scaler = StandardScaler()
todos_batches_treino_scaled = scaler.fit(todos_batches_treino)


# FAZENDO O TRANSFORM PARA CADA BATCH (TANTO TREINO QUANTO TESTE):

for i in batches_utilizados:
    temporario = globals()['dados_batch_'+str(i)].copy()
    escalado = pd.DataFrame(scaler.transform(temporario), columns = colunas).copy()
    
    globals()['dados_batch_'+str(i)] = escalado 

In [ ]:
len_batches = globals()['dados_batch_' + str(batches_treino[0])].shape[0]
len_batches

In [ ]:
# Criando os objetos X (input) e Y (output) para o treino:

X_treino = []
Y_treino = []

# Especifique o tamanho da janela de treino:
T = 20

for i in batches_treino:
    
    globals()['X_batch_'+str(i)] = globals()['dados_batch_' + str(i)][colunas_input].values

    
    globals()['Y_batch_'+str(i)] = globals()['dados_batch_' + str(i)][colunas_output].values
    

    for t in range(len_batches-T):
        x = globals()['X_batch_'+str(i)][t:t+T]
        X_treino.append(x)

        y =  globals()['Y_batch_'+str(i)][t+T]
        Y_treino.append(y)

# Observe que X é uma 'tabelona' com TODAS as janelas de TODOS os batches, e Y é o output de cada uma destas janelas.
np.array(X_treino)
X_treino = np.array(X_treino).reshape(-1,T,qntd_features_input)
Y_treino = np.array(Y_treino).reshape(-1,1)

In [ ]:
# Criando os objetos X (input) e Y (output) para o teste:

X_teste = []
Y_teste = []

for i in batches_teste:
    
    globals()['X_batch_'+str(i)] = globals()['dados_batch_' + str(i)][colunas_input].values

    
    globals()['Y_batch_'+str(i)] = globals()['dados_batch_' + str(i)][colunas_output].values
    

    for t in range(len_batches-T):
        x = globals()['X_batch_'+str(i)][t:t+T]
        X_teste.append(x)

        y =  globals()['Y_batch_'+str(i)][t+T]
        Y_teste.append(y)

X_teste = np.array(X_teste).reshape(-1,T,qntd_features_input)
Y_teste = np.array(Y_teste).reshape(-1,1)       

# FUNÇÃO GENERALIZADORA DE MODELO

In [ ]:
def Modelo_generalizado(primeira_camada, segunda_camada, terceira_camada, numero_neuronios, T, qntd_features_input):
        
    tf.keras.backend.clear_session()
    
    i = Input(shape = (T,qntd_features_input))
    
    numero_camadas = 1
    proporções = [] 
    
    if(primeira_camada == '0'):
    
        return('Erro: primeira camada inexistente')
    
    if(terceira_camada == '0'):
    
        if(segunda_camada == '0'):
    
            numero_camadas = 1
    
        else :
            numero_camadas = 2
    else: 
        numero_camadas = 3
    
    #--------------------------------------------------------------------------------------------------
    #Caso tenhamos tres camadas    
    
    if(numero_camadas == 3):
        proporcoes = [3, 2, 1]
        
        #Definindo a primeira camada
        if(primeira_camada == 'D'):
        
            x = Dense(numero_neuronios*proporcoes[0]//sum(proporcoes))(i)
    
        elif(primeira_camada == 'S'): 
            
            x = SimpleRNN(numero_neuronios*proporcoes[0]//sum(proporcoes), return_sequences = True)(i)

        elif(primeira_camada == 'L'): 
            
            x = LSTM(numero_neuronios*proporcoes[0]//sum(proporcoes), return_sequences = True)(i)
     
        elif(primeira_camada == 'G'): 
            
            x = GRU(numero_neuronios*proporcoes[0]//sum(proporcoes), return_sequences = True)(i)
    
        elif(primeira_camada == 'E'): 
            
            x = RNN(ESNCell(numero_neuronios*proporcoes[0]//sum(proporcoes)), return_sequences = True)(i)
        
        
        #Definindo a segunda camada
        
        if(segunda_camada == 'D'):
        
            x = Dense(numero_neuronios*proporcoes[1]//sum(proporcoes))(x)
        
        elif(segunda_camada == 'S'): 
        
            x = SimpleRNN(numero_neuronios*proporcoes[1]//sum(proporcoes), return_sequences = True)(x)
        
        elif(segunda_camada == 'L'): 
            
            x = LSTM(numero_neuronios*proporcoes[1]//sum(proporcoes), return_sequences = True)(x)
        
        elif(segunda_camada == 'G'): 
            
            x = GRU(numero_neuronios*proporcoes[1]//sum(proporcoes), return_sequences = True)(x)
    
        elif(segunda_camada == 'E'): 
            
            x = RNN(ESNCell(numero_neuronios*proporcoes[1]//sum(proporcoes)), return_sequences = True)(x)
    
    
        #Definindo a terceira camada
        if(terceira_camada == 'D'):
            
            x = Dense(numero_neuronios*proporcoes[2]//sum(proporcoes))(x)
    
        elif(terceira_camada == 'S'): 
            
            x = SimpleRNN(numero_neuronios*proporcoes[2]//sum(proporcoes))(x)

        elif(terceira_camada == 'L'): 
            
            x = LSTM(numero_neuronios*proporcoes[2]//sum(proporcoes))(x)
     
        elif(terceira_camada == 'G'): 
            
            x = GRU(numero_neuronios*proporcoes[2]//sum(proporcoes))(x)
    
        elif(terceira_camada == 'E'): 
            
            x = RNN(ESNCell(numero_neuronios*proporcoes[2]//sum(proporcoes)))(x)
    
    
    #--------------------------------------------------------------------------------------------------
    elif(numero_camadas == 2):
        proporcoes = [2, 1]
        
        if(primeira_camada == 'D'):
            
            x = Dense(numero_neuronios*proporcoes[0]//sum(proporcoes))(i)
    
        elif(primeira_camada == 'S'): 
            
            x = SimpleRNN(numero_neuronios*proporcoes[0]//sum(proporcoes), return_sequences = True)(i)

        elif(primeira_camada == 'L'): 
            
            x = LSTM(numero_neuronios*proporcoes[0]//sum(proporcoes), return_sequences = True)(i)
     
        elif(primeira_camada == 'G'): 
            
            x = GRU(numero_neuronios*proporcoes[0]//sum(proporcoes), return_sequences = True)(i)
    
        elif(primeira_camada == 'E'): 
            
            x = RNN(ESNCell(numero_neuronios*proporcoes[0]//sum(proporcoes)), return_sequences = True)(i)
        
        #Definindo a segunda camada
        if(segunda_camada == 'D'):
            
            x = Dense(numero_neuronios*proporcoes[1]//sum(proporcoes))(x)
    
        elif(segunda_camada == 'S'): 
            
            x = SimpleRNN(numero_neuronios*proporcoes[1]//sum(proporcoes))(x)

        elif(segunda_camada == 'L'): 
            
            x = LSTM(numero_neuronios*proporcoes[1]//sum(proporcoes))(x)
     
        elif(segunda_camada == 'G'): 
            
            x = GRU(numero_neuronios*proporcoes[1]//sum(proporcoes))(x)
    
        elif(segunda_camada == 'E'): 
            
            x = RNN(ESNCell(numero_neuronios*proporcoes[1]//sum(proporcoes)))(x)
    
    
    #--------------------------------------------------------------------------------------------------
    if(numero_camadas == 1):
        proporcoes = [1]
    
        if(primeira_camada == 'D'):
            
            x = Dense(numero_neuronios*proporcoes[0]//sum(proporcoes))(i)
    
        elif(primeira_camada == 'S'): 
            
            x = SimpleRNN(numero_neuronios*proporcoes[0]//sum(proporcoes))(i)

        elif(primeira_camada == 'L'): 
            
            x = LSTM(numero_neuronios*proporcoes[0]//sum(proporcoes))(i)
     
        elif(primeira_camada == 'G'): 
            
            x = GRU(numero_neuronios*proporcoes[0]//sum(proporcoes))(i)
    
        elif(primeira_camada == 'E'): 
            
            x = RNN(ESNCell(numero_neuronios*proporcoes[0]//sum(proporcoes)))(i)
    
    
    #--------------------------------------------------------------------------------------------------
    x = Dense(1)(x)
    
    model = Model(i,x)
    
    model.compile(loss = 'mse', optimizer = Adam(learning_rate = 0.01))

    return(model)



## FUNÇÃO TREINADORA DE MODELO GENERALIZADO

In [ ]:
def Treinadora_modelo(model, X_treino, Y_treino, X_teste, Y_teste):
    
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=20, restore_best_weights=True)
    r = model.fit(X_treino, Y_treino, epochs = 100, validation_data=(X_teste, Y_teste), callbacks=[early_stop])
    
    return(r)

## FUNÇÃO DE PREDIÇÕES DO MODELO GENERALIZADO

In [ ]:
# Tomar cuidado, na função a seguir, com o passo de atualização das janelas. Do jeito que o código está escrito, as janelas de
# input DEVEM conter 4 features, sendo a última delas a Penicillin concentration(P:g/L). Caso o usuário queira alterar o número
# de features, esta parte deve ser revisada.

def Predicao_modelo(inputs_batch, outputs_batch, model):
    
    start_time = time.time()

    validation_target  = [0]*len(batches_teste)

    final_predictions = []
    
    for k in range(len(batches_teste)):

        validation_predictions = []
        validation_target[k] = outputs_batch[k]

        # first validation input
        i = 0
        last_x = inputs_batch[k][0]

        while len(validation_predictions) < len(validation_target[k])-1:
            p = model.predict(last_x.reshape(1, -1, qntd_features_input))[0,0] # O last_x é um array de ordem 2. Por isso, precisamos fazer o reshape para que seja de ordem 3

            # update the predictions list
            validation_predictions.append(p)

            # make the new input
            # fazer só para a penicilina 

            last_x = np.roll(last_x, -1, axis = 0)

            i += 1


            last_x[-1][0] = inputs_batch[k][i][-1][0] # 'Carbon evolution rate(CER:g/h)'
            last_x[-1][1] = inputs_batch[k][i][-1][1] # 'Dissolved oxygen concentration(DO2:mg/L)'
            last_x[-1][2] = inputs_batch[k][i][-1][2] # 'Substrate concentration(S:g/L)'
            last_x[-1][3] = p                         # 'Penicillin concentration(P:g/L)'


        final_predictions.append(validation_predictions)

        print('%d%%' %(100*((k+1)/len(batches_teste))), sep=' ', end='\r')

    print("\n--- %s seconds ---" % (time.time() - start_time))
    
    return(final_predictions, validation_target)


## Avaliação dos modelos gerados

In [ ]:
def Avaliacao_modelos(batches_teste, validation_target, final_predictions):
    #precisamos remover o ultimo elemento de cada validation_target[j]. Para evitar um erro anterior, tivemos de fazer cada
    #final_predictions_geral[i][j] um elemento mais curto
    i=0
    dict_preds_multistep = {}
    for batch in batches_teste:
        dict_preds_multistep[batch] = final_predictions[i]
        i+=1

    i=0
    dict_valid = {}
    for batch in batches_teste:
        dict_valid[batch] = validation_target[i]
        i+=1

    dict_rmse = {}
    for batch in batches_teste:
        dict_rmse[batch] = metrics.mean_squared_error(dict_preds_multistep[batch], dict_valid[batch][:-1], squared=False)
    return (dict_rmse)


## Registro dos resultados em um arquivo excel + plots em uma pasta

In [ ]:
def Salva_resultados(descricao_modelo, r_replicas, final_predictions_replicas, dict_rmse_replicas, model_replicas, quantidade_replicas_por_modelo, modelo_idx):

    # CRIA TODOS OS DATAFRAMES QUE PRECISAMOS:
    
    # Sheet 1: informações sobre batches de treino e de teste; features utilizados

    resultados_modelo_sheet1 = {}

    resultados_modelo_sheet1['batches_utilizados'] = [str(batches_utilizados)]
    resultados_modelo_sheet1['batches_treino'] = [str(batches_treino)]
    resultados_modelo_sheet1['número de batches no treino'] = [str(len(batches_treino))]
    resultados_modelo_sheet1['batches_teste'] = [str(batches_teste)]
    resultados_modelo_sheet1['número de batches no teste'] = [str(len(batches_teste))]
    resultados_modelo_sheet1['número de dados (linhas) p/ cada batch'] = [str(len_batches)]
    resultados_modelo_sheet1['número de horas de experimento p/ cada batch'] = [str(batches_list[batches_treino[0]]['Time (h)'].iloc[-1])]

    resultados_modelo_sheet1['colunas_input'] = [str(colunas_input)]
    resultados_modelo_sheet1['colunas_output'] = [str(colunas_output)]
    resultados_modelo_sheet1['tamanho janela'] = [str(T)]

    df_resultados_modelo_sheet1 = pd.DataFrame(resultados_modelo_sheet1).T

    #------------
    # Sheet 2: resumo da estrutura do modelo de rede neural

    stringlist = []
    y = model_replicas[0].summary(line_length=70, print_fn=lambda x: stringlist.append(x))   #fonte: https://stackoverflow.com/questions/65873433/get-keras-model-summary-as-table
    df1_resultados_modelo_sheet2 = pd.DataFrame(stringlist, columns = ['modelo: ' + str(descricao_modelo)]).drop([3,10]) #dropei as linhas 3 e 10 ques estavam bugando o excel (não tinha nada de relevante nelas)

    optimizer_config = model_replicas[0].optimizer.get_config()
    parametros = [model_replicas[0].loss, optimizer_config['name'], optimizer_config['learning_rate'], len(r_replicas[0].history['loss']) ] #fonte: https://stackoverflow.com/questions/60212925/is-there-a-keras-function-to-obtain-the-compile-options
    df2_resultados_modelo_sheet2 = pd.DataFrame(parametros, ['Loss function', 'Optimizer', 'Learning rate', 'Epochs efetuadas (máx: 100)'])

    #------------
    # Sheet 3: detalhes de cada layer do modelo de rede neural (input; hidden layers; output)

    qtde_layers = len(model_replicas[0].layers)
    detalhes_layers = [0]*(qtde_layers)
    for i in range(qtde_layers):
        configs_layer = pd.DataFrame(np.array([[str(x) for x in model_replicas[0].layers[i].get_config().values()]]),
                                     columns=[str(x) for x in model_replicas[0].layers[i].get_config().keys()], index = ['layer'+str(i)]).T
        detalhes_layers[i] = configs_layer
    
    #------------
    # Sheet 4: loss e val_loss
    
    df_loss = [0]*(quantidade_replicas_por_modelo)
    df_val_loss = [0]*(quantidade_replicas_por_modelo)
    
    for i in range(quantidade_replicas_por_modelo):
        df_loss[i] = pd.DataFrame(r_replicas[i].history['loss'], columns = ['loss replica ' + str(i+1)])
        df_val_loss[i] = pd.DataFrame(r_replicas[i].history['val_loss'], columns = ['val_loss replica ' + str(i+1)])

    #------------
    # Sheet 5: resultados das predições one-step
    
    df_Y_treino = pd.DataFrame(Y_treino.flatten(), columns = ['Y_treino'])
    df_predicoes_onestep_treino_replicas = [0]*(quantidade_replicas_por_modelo)
    for i in range(quantidade_replicas_por_modelo):
        df_predicoes_onestep_treino_replicas[i] = pd.DataFrame(model_replicas[i].predict(X_treino).flatten(), columns = ['predicoes treino one-step replica ' + str(i+1)])
    
    df_Y_teste = pd.DataFrame(Y_teste.flatten(), columns = ['Y_teste'])
    df_predicoes_onestep_teste_replicas = [0]*(quantidade_replicas_por_modelo)
    for i in range(quantidade_replicas_por_modelo):
        df_predicoes_onestep_teste_replicas[i] = pd.DataFrame(model_replicas[i].predict(X_teste).flatten(), columns = ['predicoes teste one-step replica ' + str(i+1)])

    #------------
    # Sheet 6: resultados das predições multi-step
    
    df_predicoes_multistep_replicas = [0]*(quantidade_replicas_por_modelo)
    
    for i in range(quantidade_replicas_por_modelo):
        
        df_predicoes_multistep_batch = [0]*(len(batches_teste))
        
        passador = 0
        
        for j in range(len(batches_teste)):
            
            df_predicoes_multistep_batch[passador] = pd.DataFrame(np.array(final_predictions_replicas[i][j]).flatten(),  columns = ['predicoes multi-step replica ' + str(i+1) + ', batch ' + str(batches_teste[j])])
            passador += 1
            
        df_predicoes_multistep_replicas[i] = df_predicoes_multistep_batch
    
    #------------
    # Sheet 7: RMSEs
    
    df_rmses_replicas = [0]*(quantidade_replicas_por_modelo)
    
    for i in range(quantidade_replicas_por_modelo):
        rmses_replica = dict_rmse_replicas[i]   
        df_rmses = pd.DataFrame(rmses_replica, index = ['replica ' + str(i+1)])
        df_rmses['média'] = df_rmses.T.mean()
        df_rmses = df_rmses.T
        
        df_rmses_replicas[i] = df_rmses
        
        del df_rmses
    
    #------------------------------------------------------------------------------------------------------------------------

    #SALVA TODOS OS DATAFRAMES EM UM EXCEL
    
    #path_pasta = "Modelo"+str(modelo_idx)+'_'+descricao_modelo[0]+'_'+descricao_modelo[1]+'_'+descricao_modelo[2]+'_'+str(descricao_modelo[3]) 
    #os.mkdir(path_pasta)
    
    with pd.ExcelWriter("Modelo"+str(modelo_idx)+'_'+descricao_modelo[0]+'_'+descricao_modelo[1]+'_'+descricao_modelo[2]+'_'+str(descricao_modelo[3])+'_resultados.xlsx', engine="openpyxl") as writer:
        #, if_sheet_exists="overlay"
        #Sheet 1
        df_resultados_modelo_sheet1.to_excel(writer, sheet_name="Info inputs")

        #Sheet 2
        df1_resultados_modelo_sheet2.to_excel(writer, sheet_name="Resumo modelo")  
        df2_resultados_modelo_sheet2.to_excel(writer, sheet_name="Resumo modelo", startrow=16) 

        #Sheet 3
        for i in range(qtde_layers):
            detalhes_layers[i].to_excel(writer, sheet_name="Detalhes layers", startcol=i*4) 

        #Sheet 4
        for i in range(quantidade_replicas_por_modelo):
            df_loss[i].to_excel(writer, sheet_name="loss e val_loss", startcol = i*6) 
            df_val_loss[i].to_excel(writer, sheet_name="loss e val_loss", startcol = (i*6)+3) 

        #Sheet 5
        df_Y_treino.to_excel(writer, sheet_name="predicoes one-step", startcol = 0) 
        for i in range(quantidade_replicas_por_modelo):
            df_predicoes_onestep_treino_replicas[i].to_excel(writer, sheet_name="predicoes one-step", startcol = (i+1)*3)     

        df_Y_teste.to_excel(writer, sheet_name="predicoes one-step", startcol = (quantidade_replicas_por_modelo+1)*3) 
        for i in range(quantidade_replicas_por_modelo):
            df_predicoes_onestep_teste_replicas[i].to_excel(writer, sheet_name="predicoes one-step", startcol = (quantidade_replicas_por_modelo+i+2)*3)     

        #Sheet 6
        for i in range(quantidade_replicas_por_modelo):

            for j in range(len(batches_teste)):

                df_predicoes_multistep_replicas[i][j].to_excel(writer, sheet_name="predicoes multi-step", startcol = (3*i*len(batches_teste) + j*3))     

        #Sheet 7
        for i in range(quantidade_replicas_por_modelo):
            df_rmses_replicas[i].to_excel(writer, sheet_name="RMSEs", startcol = (i+1)*3)     


    #fonte: https://pandas.pydata.org/docs/reference/api/pandas.ExcelWriter.html

## Computa-se todos os modelos requeridos - tomando atenção para respeitar as convenções criadas abaixo:

In [ ]:
# recortar em cada batch para fazer a retroalimentação
# len_batches-T #número de janelas em cada batch

N_inputs_batch = len_batches-T

primeiro_input = 0

#Cada elemento de inputs_batch e de outputs_batch será composto de uma matriz ou lista que contém todos os inputs ou outputs
#de determinado batch. Estas variáveis servem apenas para separarmos batch por batch.
inputs_batch = []
outputs_batch = []

for i in range(len(batches_teste)):
    
    inputs_batch.append(X_teste[primeiro_input:primeiro_input+N_inputs_batch])
    outputs_batch.append(Y_teste[primeiro_input:primeiro_input+N_inputs_batch])

    primeiro_input += N_inputs_batch
    
tipos_camadas = ['0', 'S', 'L', 'G', 'E']
combinacoes_camadas = [[1,1,1], [2,2,2], [3,3,3], [4,4,4], 
                       [1,1,0], [2,2,0], [3,3,0], [4,4,0], 
                       [1,0,0], [2,0,0], [3,0,0], [4,0,0]]
                      
      
#modelos = [ ['D', '0', '0', 1] ]  # Lista dos modelos que serão testados

modelos = []
num_tot_neuronios = 30

for i in range(len(combinacoes_camadas)):
    modelos.append([tipos_camadas[combinacoes_camadas[i][0]], tipos_camadas[combinacoes_camadas[i][1]], tipos_camadas[combinacoes_camadas[i][2]], num_tot_neuronios]) 

# Cada elemento do objeto "modelos" é a descrição da estrutura do modelo a ser executado. Esta descrição obedece a seguinte
# sequência: [primeira_camada, segunda_camada, terceira_camada, numero_neuronios]. O numero_neuronios é o número total de
# neurônios utilizados na rede neural. As camadas podem ser "S", "L", "G" ou "E", representado cada RNN estudada, ou, ainda,
# "0", representando a ausência de determinada camada na rede neural. Ex: [2,2,0,30] trata de uma rede neural composta de 
# duas camadas do tipo LSTM, e que contém 30 neurônios ao todo. Lembrando que a distribuição de neurônios por camada é de 3:2:1
# quando há 3 camadas, e 2:1 quando há 2 camadas.
    
quantidade_replicas_por_modelo = 3
modelos_idx = [m for m in range(12)]
passador_modelos = 0

for i in modelos:

    primeira_camada, segunda_camada, terceira_camada, numero_neuronios = i[0], i[1], i[2], i[3]

    #Guardarei o 'r', 'final_predictions' e 'dict_rmse' de cada réplica para poder colocar tudo em um só arquivo excel
    model_replicas = [0]*(quantidade_replicas_por_modelo)  # Só preciso desse model_replicas para poder colocar as predições one-step de cada modelo no excel
    r_replicas = [0]*(quantidade_replicas_por_modelo)
    final_predictions_replicas = [0]*(quantidade_replicas_por_modelo)
    dict_rmse_replicas = [0]*(quantidade_replicas_por_modelo)

    for j in range(quantidade_replicas_por_modelo):
        set_seed(j)
        
        #Cria o modelo
        model_replicas[j] = Modelo_generalizado(primeira_camada, segunda_camada, terceira_camada, numero_neuronios, T, qntd_features_input)
        
        r_replicas[j] = Treinadora_modelo(model_replicas[j], X_treino, Y_treino, X_teste, Y_teste)
        
        print('acabou r_replicas')
        final_predictions_replicas[j], validation_target = Predicao_modelo(inputs_batch, outputs_batch, model_replicas[j])

        dict_rmse_replicas[j] = Avaliacao_modelos(batches_teste, validation_target, final_predictions_replicas[j])
        
    modelo_idx = modelos_idx[passador_modelos]
    Salva_resultados(i, r_replicas, final_predictions_replicas, dict_rmse_replicas, model_replicas, quantidade_replicas_por_modelo, modelo_idx)

    passador_modelos += 1

# ----------------------------------------